In [ ]:
# Automatically reload modules after executing each cell.
%load_ext autoreload
%autoreload 2

In [ ]:
# General imports
import os
from scipy import stats

# Utility imports
from utils.losses import *
from utils.plotting import *
from utils.training import *

np.random.seed(666) # Need to do more to ensure data is the same across runs.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # pick a number < 4 on ML4HEP; < 3 on Voltan 
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# BCE $C$ Parametrizations

In [ ]:
# Experiment parameters
num = 1
reps = 10

# Data generation
N = 10**6
bkgd = stats.norm(-0.1, 1)
sgnl = stats.norm(0.1, 1)
data_args = make_data(bkgd, sgnl, N) + [N]

# Model parameters
params_1 = {'data_args': data_args, 'loss':bce}
params_2 = {'data_args': data_args, 'loss':square_bce, 'output':'linear'}
params_3 = {'data_args': data_args, 'loss':exp_bce, 'output':'linear'}

filestr_1 = 'models/bce_c_param/set_' + str(num) + '/linear/model_{}.h5'
filestr_2 = 'models/bce_c_param/set_' + str(num) + '/square/model_{}.h5'
filestr_3 = 'models/bce_c_param/set_' + str(num) + '/exp/model_{}.h5'

In [ ]:
%%time
# Train and save models

for i in range(reps):
    print(i, end = ' ')
    model_1 = train(**params_1)
    model_2 = train(**params_2)
    model_3 = train(**params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

In [ ]:
# Get model likelihood ratios.
lrs_1 = [None] * reps
lrs_2 = [None] * reps
lrs_3 = [None] * reps
for i in range(reps):
    model_1 = create_model(**params_1)
    model_2 = create_model(**params_2)
    model_3 = create_model(**params_3)
    model_1.load_weights(filestr_1.format(i))
    model_2.load_weights(filestr_2.format(i))
    model_3.load_weights(filestr_3.format(i))
    
    lrs_1[i] = odds_lr(model_1)
    lrs_2[i] = square_odds_lr(model_2)
    lrs_3[i] = exp_odds_lr(model_3)

In [ ]:
# Get average predictions and errors. Add on the labels for plotting.
lr_1 = avg_lr(get_preds(lrs_1)) + ('BCE (linear)',)
lr_2 = avg_lr(get_preds(lrs_2)) + ('BCE (square)',)
lr_3 = avg_lr(get_preds(lrs_3)) + ('BCE (exponential)',)

lrr_1 = avg_lrr(get_preds(lrs_1)) + ('BCE (linear)',)
lrr_2 = avg_lrr(get_preds(lrs_2)) + ('BCE (square)',)
lrr_3 = avg_lrr(get_preds(lrs_3)) + ('BCE (exponential)',)

In [ ]:
lr_plot([lr_1, lr_2, lr_3], 
        r'BCE $C$ Parametrizations',
        filename='plots/bce_c_param/set_{}/lrs.png'.format(num))

In [ ]:
lrr_plot([lrr_1, lrr_2, lrr_3], 
         r'BCE $C$ Parametrizations',
         filename='plots/bce_c_param/set_{}/lrrs.png'.format(num))

# MSE $C$ Parametrizations

In [ ]:
num = 1
reps = 10

# Model parameters
params_1 = {'loss':mse}
params_2 = {'loss':square_mse, 'output':'linear'}
params_3 = {'loss':exp_mse, 'output':'linear'}

filestr_1 = 'models/mse_c_param/set_' + str(num) + '/linear/model_{}.h5'
filestr_2 = 'models/mse_c_param/set_' + str(num) + '/square/model_{}.h5'
filestr_3 = 'models/mse_c_param/set_' + str(num) + '/exp/model_{}.h5'

In [ ]:
%%time
# Train and save models

for i in range(reps):
    print(i, end = ' ')
    model_1 = train(**params_1)
    model_2 = train(**params_2)
    model_3 = train(**params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

In [ ]:
# Get model likelihood ratios.
lrs_1 = [None] * reps
lrs_2 = [None] * reps
lrs_3 = [None] * reps
for i in range(reps):
    model_1 = create_model(**params_1)
    model_2 = create_model(**params_2)
    model_3 = create_model(**params_3)
    model_1.load_weights(filestr_1.format(i))
    model_2.load_weights(filestr_2.format(i))
    model_3.load_weights(filestr_3.format(i))
    
    lrs_1[i] = odds_lr(model_1)
    lrs_2[i] = square_odds_lr(model_2)
    lrs_3[i] = exp_odds_lr(model_3)

In [ ]:
# Get average predictions and errors. Add on the labels for plotting.
lr_1 = avg_lr(get_preds(lrs_1)) + ('MSE (linear)',)
lr_2 = avg_lr(get_preds(lrs_2)) + ('MSE (square)',)
lr_3 = avg_lr(get_preds(lrs_3)) + ('MSE (exponential)',)

lrr_1 = avg_lrr(get_preds(lrs_1)) + ('MSE (linear)',)
lrr_2 = avg_lrr(get_preds(lrs_2)) + ('MSE (square)',)
lrr_3 = avg_lrr(get_preds(lrs_3)) + ('MSE (exponential)',)

In [ ]:
lr_plot([lr_1, lr_2, lr_3], 
        r'MSE $C$ Parametrizations',
        filename='plots/mse_c_param/set_{}/lrs.png'.format(num))

In [ ]:
lrr_plot([lrr_1, lrr_2, lrr_3], 
         r'MSE $C$ Parametrizations',
         filename='plots/mse_c_param/set_{}/lrrs.png'.format(num))

# MSE $A/B$ Parametrization

In [ ]:
num = 0
reps = 20

ps = np.round(np.linspace(-4, 4, 161), 2)
filestr = 'models/mse_ab_param/set_' + str(num) + '/model_{}_{}.h5'

In [ ]:
# Train models
for p in ps:
    print('===================================================\n{}'.format(p))
    params = {'loss': get_mse(p)}
    for i in range(reps):
        print(i, end = ' ')
        model = train(**params)
        print()
        model.save_weights(filestr.format(p, i))

In [ ]:
# Get model likelihood ratios.
lrs = {}
for p in ps:
    print(p)
    lrs[p] = [None] * reps
    params = {'loss': get_mse(p)}
    for i in range(reps):
        model = create_model(**params)
        model.load_weights(filestr.format(p, i))
        lrs[p][i] = pow_odds_lr(model, p) #### FIX THIS
        print(i, end = ' ')
    print()

In [ ]:
mae_avg = []
mae_err = []

for p in ps:
    maes = [mae(lr) for lr in lrs[p]]
    mae_avg += [np.mean(maes)]
    mae_err += [np.std(maes)]
    print(p, '\t', mae_avg[-1])

In [ ]:
mae_avg = np.array(mae_avg)
mae_err = np.array(mae_err)

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

plt.plot(ps, mae_avg, c='blue')

plt.minorticks_on()
plt.tick_params(direction='in', which='both',length=5)
plt.ylim(0, 0.5)

plt.ylabel('Mean Absolute Error')
plt.xlabel(r'$p$')

plt.title(r"$\mu_{\rm{sgnl}}="+str(mu)+r", \mu_{\rm{bkgd}}="+str(-mu)+r"$",
          loc="right",
          fontsize=20);
plt.title(r"MSE $A/B$ Parametrization",loc="left",fontsize=20);
plt.savefig('plots/mse_ab_param/set_{}/maes.png'.format(num), 
            dpi=1200, 
            bbox_inches='tight')

# MLC $C$ Parametrizations

In [ ]:
### Generate the data.
N = 10**6
r = 5
lam = 1

# Background is Gamma(r, λ). Signal is Gamma(r + 1, λ)
bkgd = np.random.gamma(r, lam, N)
sgnl = np.random.gamma(r + 1, lam, N)
D = np.concatenate([bkgd, sgnl])
X = (D - np.mean(D)) / np.std(D)
y = np.concatenate([np.zeros(N), np.ones(N)])

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
def lr(x):
    #return np.exp(-10 + 11) * (10/11)**x
    #return np.exp(-(1/(2 * sigma**2)) * ( (x - mu)**2 - (x + mu)**2))
    r_1 = r
    r_2 = r + 1
    v_1 = lam
    v_2 = lam
    return v_2**r_2 * np.math.factorial(r_1 - 1) * x**(r_2 - r_1) * \
           np.exp(-(v_2 - v_1)*x) / (v_1**r_1 * np.math.factorial(r_2 - 1))

In [ ]:
num = 4
reps = 100

# Model parameters
params_1 = {'loss':mlc, 'output':'relu'}
params_2 = {'loss':square_mlc, 'output':'linear'}
params_3 = {'loss':exp_mlc, 'output':'linear'}

filestr_1 = 'models/mlc_c_param/set_' + str(num) + '/linear/model_{}.h5'
filestr_2 = 'models/mlc_c_param/set_' + str(num) + '/square/model_{}.h5'
filestr_3 = 'models/mlc_c_param/set_' + str(num) + '/exp/model_{}.h5'

In [ ]:
%%time
# Train and save models

for i in range(reps):
    print(i, end = ' ')
    model_1 = train(**params_1)
    model_2 = train(**params_2)
    model_3 = train(**params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

In [ ]:
# Get model likelihood ratios.
lrs_1 = [None] * reps
lrs_2 = [None] * reps
lrs_3 = [None] * reps
for i in range(reps):
    model_1 = create_model(**params_1)
    model_2 = create_model(**params_2)
    model_3 = create_model(**params_3)
    model_1.load_weights(filestr_1.format(i))
    model_2.load_weights(filestr_2.format(i))
    model_3.load_weights(filestr_3.format(i))
    
    lrs_1[i] = pure_lr(model_1)
    lrs_2[i] = square_lr(model_2)
    lrs_3[i] = exp_lr(model_3)

In [ ]:
# Get average predictions and errors. Add on the labels for plotting.
ds = np.linspace(0, 20, 1000)
xs = (ds - np.mean(D)) / np.std(D)

lr_1 = avg_lr(get_preds(lrs_1, xs)) + ('MLC (linear)',)
lr_2 = avg_lr(get_preds(lrs_2, xs)) + ('MLC (square)',)
lr_3 = avg_lr(get_preds(lrs_3, xs)) + ('MLC (exponential)',)

lrr_1 = avg_lrr(get_preds(lrs_1, xs), ds) + ('MLC (linear)',)
lrr_2 = avg_lrr(get_preds(lrs_2, xs), ds) + ('MLC (square)',)
lrr_3 = avg_lrr(get_preds(lrs_3, xs), ds) + ('MLC (exponential)',)

In [ ]:
lr_plot([lr_1, lr_2, lr_3], 
        r'MLC $C$ Parametrizations',
        filename='plots/mlc_c_param/set_{}/lrs.png'.format(num))

In [ ]:
lrr_plot([lrr_1, lrr_2, lrr_3], 
         r'MLC $C$ Parametrizations',
         filename='plots/mlc_c_param/set_{}/lrrs.png'.format(num))

# SQR $C$ Parametrizations

In [ ]:
### Generate the data.
N = 10**6
r = 5
lam = 1

# Background is Gamma(r, λ). Signal is Gamma(r + 1, λ)
bkgd = np.random.gamma(r, lam, N)
sgnl = np.random.gamma(r + 1, lam, N)
D = np.concatenate([bkgd, sgnl])
X = (D - np.mean(D)) / np.std(D)
y = np.concatenate([np.zeros(N), np.ones(N)])

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
def lr(x):
    #return np.exp(-10 + 11) * (10/11)**x
    #return np.exp(-(1/(2 * sigma**2)) * ( (x - mu)**2 - (x + mu)**2))
    r_1 = r
    r_2 = r + 1
    v_1 = lam
    v_2 = lam
    return v_2**r_2 * np.math.factorial(r_1 - 1) * x**(r_2 - r_1) * \
           np.exp(-(v_2 - v_1)*x) / (v_1**r_1 * np.math.factorial(r_2 - 1))

In [ ]:
num = 4
reps = 100

# Model parameters
params_1 = {'loss':sqr, 'output':'relu'}
params_2 = {'loss':square_sqr, 'output':'linear'}
params_3 = {'loss':exp_sqr, 'output':'linear'}

filestr_1 = 'models/sqr_c_param/set_' + str(num) + '/linear/model_{}.h5'
filestr_2 = 'models/sqr_c_param/set_' + str(num) + '/square/model_{}.h5'
filestr_3 = 'models/sqr_c_param/set_' + str(num) + '/exp/model_{}.h5'

In [ ]:
%%time
# Train and save models

for i in range(reps):
    print(i, end = ' ')
    model_1 = train(**params_1)
    model_2 = train(**params_2)
    model_3 = train(**params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

In [ ]:
# Get model likelihood ratios.
lrs_1 = [None] * reps
lrs_2 = [None] * reps
lrs_3 = [None] * reps
for i in range(reps):
    model_1 = create_model(**params_1)
    model_2 = create_model(**params_2)
    model_3 = create_model(**params_3)
    model_1.load_weights(filestr_1.format(i))
    model_2.load_weights(filestr_2.format(i))
    model_3.load_weights(filestr_3.format(i))
    
    lrs_1[i] = pure_lr(model_1)
    lrs_2[i] = square_lr(model_2)
    lrs_3[i] = exp_lr(model_3)

In [ ]:
# Get average predictions and errors. Add on the labels for plotting.
ds = np.linspace(0, 20, 1000)
xs = (ds - np.mean(D)) / np.std(D)

lr_1 = avg_lr(get_preds(lrs_1, xs)) + ('SQR (linear)',)
lr_2 = avg_lr(get_preds(lrs_2, xs)) + ('SQR (square)',)
lr_3 = avg_lr(get_preds(lrs_3, xs)) + ('SQR (exponential)',)

lrr_1 = avg_lrr(get_preds(lrs_1, xs), ds) + ('SQR (linear)',)
lrr_2 = avg_lrr(get_preds(lrs_2, xs), ds) + ('SQR (square)',)
lrr_3 = avg_lrr(get_preds(lrs_3, xs), ds) + ('SQR (exponential)',)

In [ ]:
lr_plot([lr_1, lr_2, lr_3], 
        title=r'SQR $C$ Parametrizations',
        filename='plots/sqr_c_param/set_{}/lrs.png'.format(num))

In [ ]:
lrr_plot([lrr_1, lrr_2, lrr_3], 
         title=r'SQR $C$ Parametrizations',
         filename='plots/sqr_c_param/set_{}/lrrs.png'.format(num))

# SQR $A$/$B$ Parametrizations

In [ ]:
num = 3
reps = 20

ps = np.round(np.linspace(-2, 2, 81), 2)
sqr_filestr = 'models/sqr_ab_param/set_' + str(num) + '/linear/model_{}_{}.h5'
exp_filestr = 'models/sqr_ab_param/set_' + str(num) + '/exp/model_{}_{}.h5'

In [ ]:
# Train models
for p in ps:
    print('===================================================\n{}'.format(p))
    sqr_params = {'loss': get_sqr(p), 'output':'relu'}
    exp_params = {'loss': get_exp_sqr(p), 'output':'linear'}
    for i in range(reps):
        print(i, end = ' ')
        sqr_model = train(**sqr_params)
        exp_model = train(**exp_params)
        print()
        sqr_model.save_weights(sqr_filestr.format(p, i))
        exp_model.save_weights(exp_filestr.format(p, i))

In [ ]:
# Get model likelihood ratios.
sqr_lrs = {}
exp_lrs = {}
for p in ps:
    print(p)
    sqr_lrs[p] = [None] * reps
    exp_lrs[p] = [None] * reps
    sqr_params = {'loss': get_sqr(p), 'output':'relu'}
    exp_params = {'loss': get_exp_sqr(p), 'output':'linear'}
    for i in range(reps):
        sqr_model = create_model(**sqr_params)
        exp_model = create_model(**exp_params)
        sqr_model.load_weights(sqr_filestr.format(p, i))
        exp_model.load_weights(exp_filestr.format(p, i))
        sqr_lrs[p][i] = pow_lr(sqr_model, p)
        exp_lrs[p][i] = exp_pow_lr(exp_model, p)
        print(i, end = ' ')
    print()

In [ ]:
sqr_mae_avg = []
sqr_mae_err = []
exp_mae_avg = []
exp_mae_err = []

for p in ps:
    sqr_maes = [mae(lr) for lr in sqr_lrs[p]]
    exp_maes = [mae(lr) for lr in exp_lrs[p]]
    sqr_mae_avg += [np.mean(sqr_maes)]
    sqr_mae_err += [np.std(sqr_maes)]
    exp_mae_avg += [np.mean(exp_maes)]
    exp_mae_err += [np.std(exp_maes)]
    print(p, '\t', sqr_mae_avg[-1], '\t', exp_mae_avg[-1])
    print(p, '\t', sqr_mae_avg[-1])

In [ ]:
sqr_mae_avg = np.array(sqr_mae_avg)
sqr_mae_err = np.array(sqr_mae_err)
exp_mae_avg = np.array(exp_mae_avg)
exp_mae_err = np.array(exp_mae_err)

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

plt.plot(ps, sqr_mae_avg, c='blue', label='linear')
plt.plot(ps, exp_mae_avg, c='red', label='exponential')
plt.legend()

plt.minorticks_on()
plt.tick_params(direction='in', which='both',length=5)
plt.ylabel('Mean Absolute Error')
plt.xlabel(r'$p$')

plt.title(r"$\mu_{\rm{sgnl}}="+str(mu)+r", \mu_{\rm{bkgd}}="+str(-mu)+r"$",
          loc="right",
          fontsize=20);
plt.title(r"SQR $A/B$ Parametrization",loc="left",fontsize=20);
plt.savefig('plots/sqr_ab_param/set_{}/maes.png'.format(num), 
            dpi=1200, 
            bbox_inches='tight')

# Loss Comparisons

In [ ]:
num = 1
reps = 100
Ns = 10**np.arange(2, 8)

# Model parameters
bce_params = {'loss':bce}
mse_params = {'loss':mse}
mlc_params = {'loss':exp_mlc, 'output':'linear'}
sqr_params = {'loss':exp_sqr, 'output':'linear'}

bce_filestr = 'models/loss_comp/set_' + str(num) + '/bce/model_{}_{}.h5'
mse_filestr = 'models/loss_comp/set_' + str(num) + '/mse/model_{}_{}.h5'
mlc_filestr = 'models/loss_comp/set_' + str(num) + '/mlc/model_{}_{}.h5'
sqr_filestr = 'models/loss_comp/set_' + str(num) + '/sqr/model_{}_{}.h5'

In [ ]:
# Train models
%%time

for N in Ns:
    print('===================================================\n{}'.format(N))
    # Generate data
    bkgd = np.random.normal(-mu, sigma, N)
    sgnl = np.random.normal(mu, sigma, N)
    X = np.concatenate([bkgd, sgnl])
    y = np.concatenate([np.zeros(N), np.ones(N)])

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    for i in range(reps):
        print(i, end = ' ')
        bce_model = train(**bce_params)
        mse_model = train(**mse_params)
        mlc_model = train(**mlc_params)
        sqr_model = train(**sqr_params)
        print()
        bce_model.save_weights(bce_filestr.format(N, i))
        mse_model.save_weights(mse_filestr.format(N, i))
        mlc_model.save_weights(mlc_filestr.format(N, i))
        sqr_model.save_weights(sqr_filestr.format(N, i))

## Mean Absolute Error

In [ ]:
# Calculate mean absolute errors
bce_mae_avg = []
mse_mae_avg = []
mlc_mae_avg = []
sqr_mae_avg = []

bce_mae_err = []
mse_mae_err = []
mlc_mae_err = []
sqr_mae_err = []

for N in Ns:
    print(N)
    bce_lrs = [None] * reps
    mse_lrs = [None] * reps
    mlc_lrs = [None] * reps
    sqr_lrs = [None] * reps
    for i in range(reps):
        bce_model = create_model(**bce_params)
        bce_model.load_weights(bce_filestr.format(N, i))
        bce_lrs[i] = odds_lr(bce_model)

        mse_model = create_model(**mse_params)
        mse_model.load_weights(mse_filestr.format(N, i))
        mse_lrs[i] = odds_lr(mse_model)

        mlc_model = create_model(**mlc_params)
        mlc_model.load_weights(mlc_filestr.format(N, i))
        mlc_lrs[i] = exp_lr(mlc_model)

        sqr_model = create_model(**sqr_params)
        sqr_model.load_weights(sqr_filestr.format(N, i))
        sqr_lrs[i] = exp_lr(sqr_model)
    
    bce_maes = [mae(lr) for lr in bce_lrs]
    mse_maes = [mae(lr) for lr in mse_lrs]
    mlc_maes = [mae(lr) for lr in mlc_lrs]
    sqr_maes = [mae(lr) for lr in sqr_lrs]
    
    bce_mae_avg += [np.mean(bce_maes)]
    bce_mae_err += [np.std(bce_maes)]
    
    mse_mae_avg += [np.mean(mse_maes)]
    mse_mae_err += [np.std(mse_maes)]
    
    mlc_mae_avg += [np.mean(mlc_maes)]
    mlc_mae_err += [np.std(mlc_maes)]
    
    sqr_mae_avg += [np.mean(sqr_maes)]
    sqr_mae_err += [np.std(sqr_maes)]

bce_mae_avg = np.array(bce_mae_avg)
mse_mae_avg = np.array(mse_mae_avg)
mlc_mae_avg = np.array(mlc_mae_avg)
sqr_mae_avg = np.array(sqr_mae_avg)

bce_mae_err = np.array(bce_mae_err)
mse_mae_err = np.array(mse_mae_err)
mlc_mae_err = np.array(mlc_mae_err)
sqr_mae_err = np.array(sqr_mae_err)

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

plt.plot(Ns, bce_mae_avg, c='brown', ls=':', label='BCE')
plt.plot(Ns, mse_mae_avg, c='green', ls='--', label='MSE')
plt.plot(Ns, mlc_mae_avg, c='red', ls='--', label='MLC')
plt.plot(Ns, sqr_mae_avg, c='blue', ls='-.', label='SQR')
#plt.fill_between(Ns, bce_mae_avg - bce_mae_err, bce_mae_avg + bce_mae_err, color='brown', alpha=0.1)
#plt.fill_between(Ns, mse_mae_avg - mse_mae_err, mse_mae_avg + mse_mae_err, color='green', alpha=0.1)
#plt.fill_between(Ns, mlc_mae_avg - mlc_mae_err, mlc_mae_avg + mlc_mae_err, color='red', alpha=0.1)
#plt.fill_between(Ns, sqr_mae_avg - sqr_mae_err, sqr_mae_avg + sqr_mae_err, color='blue', alpha=0.1)
plt.legend()

plt.xscale("log", base=10)
plt.minorticks_on()
plt.tick_params(direction='in', which='both',length=5)
plt.ylabel('Mean Absolute Error')
plt.xlabel(r'$N$')

plt.title(r"$\mu_{\rm{sgnl}}="+str(mu)+r", \mu_{\rm{bkgd}}="+str(-mu)+r"$",
          loc="right",
          fontsize=20);
plt.savefig('plots/loss_comp/set_{}/maes.png'.format(num),
            dpi=1200, 
            bbox_inches='tight')

## Likelihood Ratios

In [ ]:
N = 10**6
reps = 100

In [ ]:
# Load in models.
bce_lrs = [None] * reps
mse_lrs = [None] * reps
mlc_lrs = [None] * reps
sqr_lrs = [None] * reps
for i in range(reps):
    bce_model = create_model(**bce_params)
    bce_model.load_weights(bce_filestr.format(N, i))
    bce_lrs[i] = odds_lr(bce_model)
    
    mse_model = create_model(**mse_params)
    mse_model.load_weights(mse_filestr.format(N, i))
    mse_lrs[i] = odds_lr(mse_model)
    
    mlc_model = create_model(**mlc_params)
    mlc_model.load_weights(mlc_filestr.format(N, i))
    mlc_lrs[i] = exp_lr(mlc_model)
    
    sqr_model = create_model(**sqr_params)
    sqr_model.load_weights(sqr_filestr.format(N, i))
    sqr_lrs[i] = exp_lr(sqr_model)

In [ ]:
bce_lr = avg_lr(get_preds(bce_lrs)) + ('BCE',)
mse_lr = avg_lr(get_preds(mse_lrs)) + ('MSE',)
mlc_lr = avg_lr(get_preds(mlc_lrs)) + ('MLC',)
sqr_lr = avg_lr(get_preds(sqr_lrs)) + ('SQR',)

bce_lrr = avg_lrr(get_preds(bce_lrs)) + ('BCE',)
mse_lrr = avg_lrr(get_preds(mse_lrs)) + ('MSE',)
mlc_lrr = avg_lrr(get_preds(mlc_lrs)) + ('MLC',)
sqr_lrr = avg_lrr(get_preds(sqr_lrs)) + ('SQR',)

In [ ]:
lr_plot([bce_lr, mse_lr, mlc_lr, sqr_lr], 
        filename='plots/loss_comp/set_{}/lrs.png'.format(num))

In [ ]:
lrr_plot([bce_lrr, mse_lrr, mlc_lrr, sqr_lrr], 
         filename='plots/loss_comp/set_{}/lrrs.png'.format(num))

In [ ]:
bce_mae = np.mean([mae(lr) for lr in bce_lrs])
mse_mae = np.mean([mae(lr) for lr in mse_lrs])
mlc_mae = np.mean([mae(lr) for lr in mlc_lrs])
sqr_mae = np.mean([mae(lr) for lr in sqr_lrs])

In [ ]:
print(bce_mae, mse_mae, mlc_mae, sqr_mae)

# Interpolation Test

In [ ]:
num = 5
N = 10**6
mus = np.round(np.linspace(0.1, 1.5, 15), 2)
sigma = 1

reps = 50

# Model parameters
bce_params = {'loss':bce, 'verbose':0}
mse_params = {'loss':mse, 'verbose':0}
mlc_params = {'loss':exp_mlc, 'output':'linear', 'verbose':0}
sqr_params = {'loss':exp_sqr, 'output':'linear', 'verbose':0}

bce_filestr = 'models/interp/set_' + str(num) + '/bce/model_{}_{}.h5'
mse_filestr = 'models/interp/set_' + str(num) + '/mse/model_{}_{}.h5'
mlc_filestr = 'models/interp/set_' + str(num) + '/mlc/model_{}_{}.h5'
sqr_filestr = 'models/interp/set_' + str(num) + '/sqr/model_{}_{}.h5'

In [ ]:
%%time

for mu in mus:
    print('===================================================\n{}'.format(mu))
    # Generate data
    bkgd = np.random.normal(-mu, sigma, N)
    sgnl = np.random.normal(mu, sigma, N)
    X = np.concatenate([bkgd, sgnl])
    y = np.concatenate([np.zeros(N), np.ones(N)])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    for i in range(reps):
        print(i, end = ' ')
        bce_model = train(**bce_params)
        mse_model = train(**mse_params)
        mlc_model = train(**mlc_params)
        sqr_model = train(**sqr_params)
        print()
        bce_model.save_weights(bce_filestr.format(mu, i))
        mse_model.save_weights(mse_filestr.format(mu, i))
        mlc_model.save_weights(mlc_filestr.format(mu, i))
        sqr_model.save_weights(sqr_filestr.format(mu, i))

## Expected Error

In [ ]:
# Calculate mean absolute errors
bce_mae_avg = []
mse_mae_avg = []
mlc_mae_avg = []
sqr_mae_avg = []

bce_mae_err = []
mse_mae_err = []
mlc_mae_err = []
sqr_mae_err = []

bce_mpe_avg = []
mse_mpe_avg = []
mlc_mpe_avg = []
sqr_mpe_avg = []

bce_mpe_err = []
mse_mpe_err = []
mlc_mpe_err = []
sqr_mpe_err = []

for mu in mus:
    print(mu)
    bce_lrs = [None] * reps
    mse_lrs = [None] * reps
    mlc_lrs = [None] * reps
    sqr_lrs = [None] * reps
    for i in range(reps):
        bce_model = create_model(**bce_params)
        bce_model.load_weights(bce_filestr.format(mu, i))
        bce_lrs[i] = odds_lr(bce_model)

        mse_model = create_model(**mse_params)
        mse_model.load_weights(mse_filestr.format(mu, i))
        mse_lrs[i] = odds_lr(mse_model)

        mlc_model = create_model(**mlc_params)
        mlc_model.load_weights(mlc_filestr.format(mu, i))
        mlc_lrs[i] = exp_lr(mlc_model)

        sqr_model = create_model(**sqr_params)
        sqr_model.load_weights(sqr_filestr.format(mu, i))
        sqr_lrs[i] = exp_lr(sqr_model)
    
    mae = make_mae(mu, sigma)
    bce_maes = [mae(lr) for lr in bce_lrs]
    mse_maes = [mae(lr) for lr in mse_lrs]
    mlc_maes = [mae(lr) for lr in mlc_lrs]
    sqr_maes = [mae(lr) for lr in sqr_lrs]
    
    bce_mae_avg += [np.mean(bce_maes)]
    bce_mae_err += [np.std(bce_maes)]
    
    mse_mae_avg += [np.mean(mse_maes)]
    mse_mae_err += [np.std(mse_maes)]
    
    mlc_mae_avg += [np.mean(mlc_maes)]
    mlc_mae_err += [np.std(mlc_maes)]
    
    sqr_mae_avg += [np.mean(sqr_maes)]
    sqr_mae_err += [np.std(sqr_maes)]
    
    print(bce_mae_avg[-1], mse_mae_avg[-1], mlc_mae_avg[-1], sqr_mae_avg[-1])
    
    mpe = make_mpe(mu, sigma)
    bce_mpes = [mpe(lr) for lr in bce_lrs]
    mse_mpes = [mpe(lr) for lr in mse_lrs]
    mlc_mpes = [mpe(lr) for lr in mlc_lrs]
    sqr_mpes = [mpe(lr) for lr in sqr_lrs]
    
    bce_mpe_avg += [np.mean(bce_mpes)]
    bce_mpe_err += [np.std(bce_mpes)]
    
    mse_mpe_avg += [np.mean(mse_mpes)]
    mse_mpe_err += [np.std(mse_mpes)]
    
    mlc_mpe_avg += [np.mean(mlc_mpes)]
    mlc_mpe_err += [np.std(mlc_mpes)]
    
    sqr_mpe_avg += [np.mean(sqr_mpes)]
    sqr_mpe_err += [np.std(sqr_mpes)]
    
    print(bce_mpe_avg[-1], mse_mpe_avg[-1], mlc_mpe_avg[-1], sqr_mpe_avg[-1])

bce_mae_avg = np.array(bce_mae_avg)
mse_mae_avg = np.array(mse_mae_avg)
mlc_mae_avg = np.array(mlc_mae_avg)
sqr_mae_avg = np.array(sqr_mae_avg)

bce_mae_err = np.array(bce_mae_err)
mse_mae_err = np.array(mse_mae_err)
mlc_mae_err = np.array(mlc_mae_err)
sqr_mae_err = np.array(sqr_mae_err)

bce_mpe_avg = np.array(bce_mpe_avg)
mse_mpe_avg = np.array(mse_mpe_avg)
mlc_mpe_avg = np.array(mlc_mpe_avg)
sqr_mpe_avg = np.array(sqr_mpe_avg)

bce_mpe_err = np.array(bce_mpe_err)
mse_mpe_err = np.array(mse_mpe_err)
mlc_mpe_err = np.array(mlc_mpe_err)
sqr_mpe_err = np.array(sqr_mpe_err)

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

plt.plot(mus, bce_mae_avg, c='brown', ls=':', label='BCE')
plt.plot(mus, mse_mae_avg, c='green', ls='--', label='MSE')
plt.plot(mus, mlc_mae_avg, c='red', ls='--', label='MLC')
plt.plot(mus, sqr_mae_avg, c='blue', ls='-.', label='SQR')
plt.legend()

plt.yscale("log", basey=10)
plt.minorticks_on()
plt.tick_params(direction='in', which='both',length=5)
plt.ylabel('Mean Absolute Error')
plt.xlabel(r'$\mu$')

plt.savefig('plots/interp/set_{}/maes.png'.format(num),
            dpi=1200, 
            bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

plt.plot(mus, bce_mpe_avg, c='brown', ls=':', label='BCE')
plt.plot(mus, mse_mpe_avg, c='green', ls='--', label='MSE')
plt.plot(mus, mlc_mpe_avg, c='red', ls='--', label='MLC')
plt.plot(mus, sqr_mpe_avg, c='blue', ls='-.', label='SQR')
plt.legend()
#plt.ylim(0, 5)

#plt.xscale("log", base=10)
plt.minorticks_on()
plt.tick_params(direction='in', which='both',length=5)
plt.ylabel('Mean Percent Error (\%)')
plt.xlabel(r'$\mu$')

plt.savefig('plots/interp/set_{}/mpes.png'.format(num),
            dpi=1200, 
            bbox_inches='tight')

## Single Test

In [ ]:
xs = np.linspace(-4, 4, 1000)
bce_lr = avg_lr(get_preds(bce_lrs, xs)) + ('BCE',)
mse_lr = avg_lr(get_preds(mse_lrs, xs)) + ('MSE',)
mlc_lr = avg_lr(get_preds(mlc_lrs, xs)) + ('MLC',)
sqr_lr = avg_lr(get_preds(sqr_lrs, xs)) + ('SQR',)

bce_lrr = avg_lrr(get_preds(bce_lrs, xs), xs) + ('BCE',)
mse_lrr = avg_lrr(get_preds(mse_lrs, xs), xs) + ('MSE',)
mlc_lrr = avg_lrr(get_preds(mlc_lrs, xs), xs) + ('MLC',)
sqr_lrr = avg_lrr(get_preds(sqr_lrs, xs), xs) + ('SQR',)

In [ ]:
lr_plot([bce_lr, mse_lr, mlc_lr, sqr_lr], 
        xs=xs,
        filename='plots/interp/set_{}/lrs.png'.format(num))

In [ ]:
lrr_plot([bce_lrr, mse_lrr, mlc_lrr, sqr_lrr], 
         xs=xs,
         filename='plots/interp/set_{}/lrrs.png'.format(num))

# Scratch

## 1. Loss Comparison for Gamma Data

In [ ]:
### Generate the data.
N = 10**6
r = 5
lam = 1

# Background is Gamma(r, λ). Signal is Gamma(r + 1, λ)
bkgd = np.random.gamma(r, lam, N)
sgnl = np.random.gamma(r + 1, lam, N)
D = np.concatenate([bkgd, sgnl])
X = (D - np.mean(D)) / np.std(D)
y = np.concatenate([np.zeros(N), np.ones(N)])

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
def lr(x):
    #return np.exp(-10 + 11) * (10/11)**x
    #return np.exp(-(1/(2 * sigma**2)) * ( (x - mu)**2 - (x + mu)**2))
    r_1 = r
    r_2 = r + 1
    v_1 = lam
    v_2 = lam
    return v_2**r_2 * np.math.factorial(r_1 - 1) * x**(r_2 - r_1) * \
           np.exp(-(v_2 - v_1)*x) / (v_1**r_1 * np.math.factorial(r_2 - 1))

In [ ]:
num = 4
reps = 20

# Model parameters
bce_params = {'loss':bce}
mse_params = {'loss':mse}
mlc_params = {'loss':exp_mlc, 'output':'linear'}
sqr_params = {'loss':exp_sqr, 'output':'linear'}

bce_filestr = 'models/scratch/set_' + str(num) + '/bce/model_{}.h5'
mse_filestr = 'models/scratch/set_' + str(num) + '/mse/model_{}.h5'
mlc_filestr = 'models/scratch/set_' + str(num) + '/mlc/model_{}.h5'
sqr_filestr = 'models/scratch/set_' + str(num) + '/sqr/model_{}.h5'

In [ ]:
%%time

for i in range(reps):
    print(i, end = ' ')
    bce_model = train(**bce_params)
    mse_model = train(**mse_params)
    mlc_model = train(**mlc_params)
    sqr_model = train(**sqr_params)
    print()
    bce_model.save_weights(bce_filestr.format(i))
    mse_model.save_weights(mse_filestr.format(i))
    mlc_model.save_weights(mlc_filestr.format(i))
    sqr_model.save_weights(sqr_filestr.format(i))

In [ ]:
# Load in models.
bce_lrs = [None] * reps
mse_lrs = [None] * reps
mlc_lrs = [None] * reps
sqr_lrs = [None] * reps
for i in range(reps):
    bce_model = create_model(**bce_params)
    bce_model.load_weights(bce_filestr.format(i))
    bce_lrs[i] = odds_lr(bce_model)
    
    mse_model = create_model(**mse_params)
    mse_model.load_weights(mse_filestr.format(i))
    mse_lrs[i] = odds_lr(mse_model)
    
    mlc_model = create_model(**mlc_params)
    mlc_model.load_weights(mlc_filestr.format(i))
    mlc_lrs[i] = exp_lr(mlc_model)
    
    sqr_model = create_model(**sqr_params)
    sqr_model.load_weights(sqr_filestr.format(i))
    sqr_lrs[i] = exp_lr(sqr_model)

In [ ]:
ds = np.linspace(0, 20, 1000)
xs = (ds - np.mean(D)) / np.std(D)

bce_lr = avg_lr(get_preds(bce_lrs, xs)) + ('BCE',)
mse_lr = avg_lr(get_preds(mse_lrs, xs)) + ('MSE',)
mlc_lr = avg_lr(get_preds(mlc_lrs, xs)) + ('MLC',)
sqr_lr = avg_lr(get_preds(sqr_lrs, xs)) + ('SQR',)

bce_lrr = avg_lrr(get_preds(bce_lrs, xs), ds) + ('BCE',)
mse_lrr = avg_lrr(get_preds(mse_lrs, xs), ds) + ('MSE',)
mlc_lrr = avg_lrr(get_preds(mlc_lrs, xs), ds) + ('MLC',)
sqr_lrr = avg_lrr(get_preds(sqr_lrs, xs), ds) + ('SQR',)

In [ ]:
lr_plot([bce_lr, mse_lr, mlc_lr, sqr_lr], 
        filename='plots/scratch/set_{}/lrs.png'.format(num))

In [ ]:
lrr_plot([bce_lrr, mse_lrr, mlc_lrr, sqr_lrr], 
         filename='plots/scratch/set_{}/lrrs.png'.format(num))

## 2. Test on Non-Zero-Centered Gaussians

In [ ]:
N = 10**6
mu = 10
d = 0.1
sigma = 1

# Background is Normal(μ - d, σ). Signal is Normal(μ + d, σ))
bkgd = np.random.normal(mu - d, sigma, N)
sgnl = np.random.normal(mu + d, sigma, N)
D = np.concatenate([bkgd, sgnl])
X = (D - np.mean(D)) / np.std(D)
y = np.concatenate([np.zeros(N), np.ones(N)])

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
def lr(x):
    return np.exp(-(1/(2 * sigma**2)) * ( (x - 10.1)**2 - (x - 9.9)**2))

In [ ]:
num = 3
reps = 20

# Model parameters
bce_params = {'loss':bce}
mse_params = {'loss':mse}
mlc_params = {'loss':exp_mlc, 'output':'linear'}
sqr_params = {'loss':exp_sqr, 'output':'linear'}

bce_filestr = 'models/scratch/set_' + str(num) + '/bce/model_{}.h5'
mse_filestr = 'models/scratch/set_' + str(num) + '/mse/model_{}.h5'
mlc_filestr = 'models/scratch/set_' + str(num) + '/mlc/model_{}.h5'
sqr_filestr = 'models/scratch/set_' + str(num) + '/sqr/model_{}.h5'

In [ ]:
%%time

for i in range(reps):
    print(i, end = ' ')
    bce_model = train(**bce_params)
    mse_model = train(**mse_params)
    mlc_model = train(**mlc_params)
    sqr_model = train(**sqr_params)
    print()
    bce_model.save_weights(bce_filestr.format(i))
    mse_model.save_weights(mse_filestr.format(i))
    mlc_model.save_weights(mlc_filestr.format(i))
    sqr_model.save_weights(sqr_filestr.format(i))

In [ ]:
# Load in models.
bce_lrs = [None] * reps
mse_lrs = [None] * reps
mlc_lrs = [None] * reps
sqr_lrs = [None] * reps
for i in range(reps):
    bce_model = create_model(**bce_params)
    bce_model.load_weights(bce_filestr.format(i))
    bce_lrs[i] = odds_lr(bce_model)
    
    mse_model = create_model(**mse_params)
    mse_model.load_weights(mse_filestr.format(i))
    mse_lrs[i] = odds_lr(mse_model)
    
    mlc_model = create_model(**mlc_params)
    mlc_model.load_weights(mlc_filestr.format(i))
    mlc_lrs[i] = exp_lr(mlc_model)
    
    sqr_model = create_model(**sqr_params)
    sqr_model.load_weights(sqr_filestr.format(i))
    sqr_lrs[i] = exp_lr(sqr_model)

In [ ]:
ds = np.linspace(4, 16, 1000)
xs = (ds - np.mean(D)) / np.std(D)

bce_lr = avg_lr(get_preds(bce_lrs, xs)) + ('BCE',)
mse_lr = avg_lr(get_preds(mse_lrs, xs)) + ('MSE',)
mlc_lr = avg_lr(get_preds(mlc_lrs, xs)) + ('MLC',)
sqr_lr = avg_lr(get_preds(sqr_lrs, xs)) + ('SQR',)

bce_lrr = avg_lrr(get_preds(bce_lrs, xs), ds) + ('BCE',)
mse_lrr = avg_lrr(get_preds(mse_lrs, xs), ds) + ('MSE',)
mlc_lrr = avg_lrr(get_preds(mlc_lrs, xs), ds) + ('MLC',)
sqr_lrr = avg_lrr(get_preds(sqr_lrs, xs), ds) + ('SQR',)

In [ ]:
lr_plot([bce_lr, mse_lr, mlc_lr, sqr_lr], 
        filename='plots/scratch/set_{}/lrs.png'.format(num))

In [ ]:
lrr_plot([bce_lrr, mse_lrr, mlc_lrr, sqr_lrr], 
         filename='plots/scratch/set_{}/lrrs.png'.format(num))